# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

Описание данных:
- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
data = pd.read_csv('/datasets/insurance.csv')

In [ ]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


В датасете имеется 5 столбцов с 5000 строк в каждом. Преобразуем тип данных столбцов "возраст" и "зарплата" в *int*, а также проверим данные на дубликаты.

In [ ]:
data['Возраст'] = data['Возраст'].astype('int')
data['Зарплата'] = data['Зарплата'].astype('int')

In [ ]:
data.duplicated().sum()

153

In [ ]:
data = data.drop_duplicates()

В данном шаге были загружены и подготовлены данные.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Задание:**

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

- a. Изменится. Приведите примеры матриц.
- b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.


**Ответ:** Качество линейной регрессии не меняется, но меняются веса для новых признаков (w1).

**Обоснование:**

$$ X1 = XP $$
 
$$ w1 = ((XP)^T XP)^{-1} (XP)^{T} y $$

$$ w1 = (P^T X^T XP)^{-1} X^T P^T y $$

$$ w1 = (P^T)^{-1} P^{-1} (X^T X)^{-1} P^T X^T y $$

$$ w1 = P^{-1} (X^T X)^{-1} X^T y $$

$$ w1 = P^{-1} w $$


Подставим значение в формулу предсказаний

$$ a1 = X1 w1 $$

$$ a1 = XP P^{-1} w $$

$$ a1 = X w $$

## Алгоритм преобразования

*Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.*

Проверим на практике предыдущий шаг.

Создадим квадратную матрицу P и проверим, обратима ли она:

In [ ]:
matrix_p = np.random.randint(50, size = (4,4))
matrix_p

array([[14,  1, 44, 45],
       [38, 36, 10, 47],
       [22, 49, 36, 48],
       [10,  8, 34, 21]])

In [ ]:
np.linalg.inv(matrix_p)

array([[-3.62318841e-02,  4.43143813e-02, -4.51505017e-02,
         8.16610925e-02],
       [-2.01210075e-02, -8.10143845e-03,  2.67558528e-02,
         9.20003031e-05],
       [-1.40706346e-02, -6.87566971e-03, -3.34448161e-03,
         5.31842928e-02],
       [ 4.76994512e-02, -6.88378738e-03,  1.67224080e-02,
        -7.74101374e-02]])

Чтобы обосновать, что качество линейной регрессии не изменится, рассчитаем a и a1 на наших данных.

$ a = Xw $

$ w = (X^T X)^{-1} X^T y $

$ a1 = XPw1 $

$ w1 = ((XP)^T XP)^{-1} (XP)^{T} y $

Выделим признаки:

In [ ]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

Посчитаем вектор весов и вектор предсказаний до преобразования:

In [ ]:
w = np.linalg.inv(np.dot(features.T,features)).dot(features.T).dot(target)

In [ ]:
a = features.dot(w)

Посчитаем вектор весов и вектор предсказаний после преобразования:

In [ ]:
w1 = np.linalg.inv(np.dot(features.dot(matrix_p).T,features.dot(matrix_p))).dot(features.dot(matrix_p).T).dot(target)

In [ ]:
a1 = features.dot(matrix_p).dot(w1)

In [ ]:
np.allclose(a,a1,atol=1e-7)

False

Получили, что a и a1 равны.

## Проверка алгоритма

Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

In [ ]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print("R2 метрика:", r2_score(target, predictions))

R2 метрика: 0.4302010046633359


In [ ]:
features_transformed = features.dot(matrix_p)
model = LinearRegression()
model.fit(features_transformed, target)
predictions = model.predict(features_transformed)
print("R2 метрика:", r2_score(target, predictions))

R2 метрика: 0.43020100466333666


Из полученных результатов видим, что метрика R2 не отличается.

## Вывод

В данном проекте необходимо было разработать метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. В качестве преобразования признаки были умножены на обратимую матрицу. Корректность работы алгоритма была обоснована равенством предсказаний и проверкой R2 метрики.

Таким образом при преобразовании качество модели не ухудшилось.